In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import yfinance as yf

# ----------------------------
# 1. دریافت دادههای تاریخی قیمت
# ----------------------------
def fetch_data(ticker="BTC-USD", start="2020-01-01", end="2023-12-31"):
    data = yf.download(ticker, start=start, end=end)
    return data

# ----------------------------
# 2. استخراج ویژگیهای مبتنی بر Price Action
# ----------------------------
def create_features(data):
    df = data.copy()
    
    # ویژگی ۱: الگوی شمعی (Doji)
    df['doji'] = np.where((abs(df['Open'] - df['Close']) / (df['High'] - df['Low']) < 0.1) & 
                        (df['High'] - df['Low'] > 0), 1, 0)
    
    # ویژگی ۲: شکست مقاومت/حمایت (ساده)
    df['resistance'] = df['High'].rolling(20).max()
    df['support'] = df['Low'].rolling(20).min()
    
    # ویژگی ۳: روند (Moving Average)
    df['ma_50'] = df['Close'].rolling(50).mean()
    df['ma_200'] = df['Close'].rolling(200).mean()
    
    # ویژگی ۴: تغییرات قیمت گذشته
    df['ret_1'] = df['Close'].pct_change(1)
    df['ret_3'] = df['Close'].pct_change(3)
    
    # حذف دادههای NaN
    df = df.dropna()
    
    return df

# ----------------------------
# 3. ایجاد برچسب (هدف: پیشبینی صعود/نزول)
# ----------------------------
def create_labels(df, horizon=3):
    df['label'] = np.where(df['Close'].shift(-horizon) > df['Close'], 1, 0)  # 1=صعودی, 0=نزولی
    return df

# ----------------------------
# 4. آموزش مدل
# ----------------------------
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, preds):.2f}")
    return model

# ----------------------------
# اجرای اصلی
# ----------------------------
if __name__ == "__main__":
    # دریافت دادهها
    data = fetch_data(ticker="BTC-USD")
    
    # ایجاد ویژگیها و برچسبها
    df = create_features(data)
    df = create_labels(df, horizon=3)
    
    # انتخاب دادهها
    features = ['doji', 'resistance', 'support', 'ma_50', 'ma_200', 'ret_1', 'ret_3']
    X = df[features]
    y = df['label']
    data.to_excel('ResultPrice.xlsx')
    Result =pd.read_excel('ResultPrice.xlsx')
    print(Result)
    # آموزش مدل
    model = train_model(X, y)

[*********************100%***********************]  1 of 1 completed


                    Price         Close          High           Low  \
0                  Ticker       BTC-USD       BTC-USD       BTC-USD   
1                    Date           NaN           NaN           NaN   
2     2020-01-01 00:00:00   7200.174316   7254.330566   7174.944336   
3     2020-01-02 00:00:00   6985.470215   7212.155273    6935.27002   
4     2020-01-03 00:00:00   7344.884277   7413.715332   6914.996094   
...                   ...           ...           ...           ...   
1457  2023-12-26 00:00:00  42520.402344  43603.175781  41676.488281   
1458  2023-12-27 00:00:00  43442.855469  43683.160156  42167.582031   
1459  2023-12-28 00:00:00  42627.855469   43804.78125  42318.550781   
1460  2023-12-29 00:00:00  42099.402344  43124.324219    41424.0625   
1461  2023-12-30 00:00:00  42156.902344     42584.125  41556.226562   

              Open       Volume  
0          BTC-USD      BTC-USD  
1              NaN          NaN  
2       7194.89209  18565664997  
3       720